<a href="https://colab.research.google.com/github/gpasxos/large-scale-optimization/blob/main/ch06_equality_ctr_newton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newton's Method for Equality-Constrained Optimization

We solve problems of the form:
$$\min_x f(x) \quad \text{subject to} \quad Ax = b$$

## KKT System for Newton Step

At each iteration, we find the Newton step $\Delta x$ and dual variables $\nu$ by solving the KKT system:

$$\begin{bmatrix} \nabla^2 f(x) & A^T \\ A & 0 \end{bmatrix} \begin{bmatrix} \Delta x \\ \nu \end{bmatrix} = \begin{bmatrix} -\nabla f(x) \\ b - Ax \end{bmatrix}$$

**Key properties:**
- If $Ax^{(k)} = b$ (feasible), then $A\Delta x = 0$, so $x^{(k+1)} = x^{(k)} + \eta \Delta x$ remains feasible for any $\eta$
- For infeasible start, the right-hand side $b - Ax$ drives the iterates toward feasibility
- The dual variables $\nu$ are the Lagrange multipliers at convergence

## Stopping Criterion

We check both:
1. **Newton decrement**: $\lambda^2/2 \leq \epsilon$ (optimality)
2. **Constraint violation**: $\|Ax - b\| < \epsilon$ (feasibility)

## Demo Problem

We minimize a quadratic $f(x) = \frac{1}{2}x^T Q x + c^T x$ subject to $Ax = b$ with:
- $n = 10$ variables, $p = 3$ equality constraints
- Random positive definite $Q$
- Starting from an infeasible point $x_0 = 0$

In [1]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve

def newton_equality_constrained(f, grad_f, hess_f, A, b, x0, tol=1e-10, max_iter=100, alpha=0.25, beta=0.5):
    """
    Newton's method for equality-constrained optimization.

    min f(x) s.t. Ax = b
    """
    x = np.array(x0, dtype=float)
    n = len(x)
    p = len(b)

    # Check initial feasibility
    residual = A @ x - b
    if np.linalg.norm(residual) > 1e-10:
        print("Warning: initial point not feasible, using infeasible-start Newton")

    history = {'f': [f(x)], 'newton_decrement': [], 'constraint_violation': [np.linalg.norm(residual)]}

    for k in range(max_iter):
        g = grad_f(x)
        H = hess_f(x)
        residual = A @ x - b

        # Form and solve KKT system
        # [H A^T] [dx] [-g ]
        # [A 0 ] [nu] = [b - Ax ]
        KKT = np.block([
            [H, A.T],
            [A, np.zeros((p, p))]
        ])
        rhs = np.concatenate([-g, -residual])

        try:
            lu, piv = lu_factor(KKT)
            sol = lu_solve((lu, piv), rhs)
        except np.linalg.LinAlgError:
            print("KKT system singular")
            break

        dx = sol[:n]
        nu = sol[n:]

        # Newton decrement
        lambda_sq = -g @ dx
        if lambda_sq < 0:
            lambda_sq = abs(lambda_sq) # Numerical issue
        newton_decrement = np.sqrt(lambda_sq)
        history['newton_decrement'].append(newton_decrement)

        # Check convergence
        if lambda_sq / 2 <= tol and np.linalg.norm(residual) < tol:
            print(f"Converged at iteration {k}")
            break

        # Backtracking line search
        eta = 1.0
        f_x = f(x)

        # Merit function: f(x) + penalty * ||Ax - b||
        penalty = 1.0
        merit = lambda z: f(z) + penalty * np.linalg.norm(A @ z - b)
        merit_x = merit(x)

        while merit(x + eta * dx) > merit_x - alpha * eta * lambda_sq:
            eta *= beta
            if eta < 1e-16:
                break

        x = x + eta * dx
        history['f'].append(f(x))
        history['constraint_violation'].append(np.linalg.norm(A @ x - b))

        print(f"Iter {k}: f = {f(x):.6e}, lambda = {newton_decrement:.2e}, "f"||Ax-b|| = {np.linalg.norm(A @ x - b):.2e}, eta = {eta:.4f}")

    return x, nu, history

def demo_equality_constrained():
    """Demo: minimize quadratic subject to linear equality constraints."""
    np.random.seed(42)
    n, p = 10, 3

    # Objective: f(x) = 0.5 * x^T Q x + c^T x
    Q = np.eye(n) + 0.5 * np.random.randn(n, n)
    Q = Q @ Q.T # Make positive definite
    c = np.random.randn(n)

    f = lambda x: 0.5 * x @ Q @ x + c @ x
    grad_f = lambda x: Q @ x + c
    hess_f = lambda x: Q

    # Constraints: Ax = b
    A = np.random.randn(p, n)
    x_feas = np.random.randn(n)
    b = A @ x_feas # Ensure feasibility is possible

    # Initial point (may not be feasible)
    x0 = np.zeros(n)

    print("="*60)
    print("Newton for Equality-Constrained Optimization")
    print("="*60)

    x_opt, nu_opt, history = newton_equality_constrained(
        f, grad_f, hess_f, A, b, x0
    )

    print(f"\nOptimal x (first 5): {x_opt[:5]}")
    print(f"Optimal multipliers: {nu_opt}")
    print(f"Final constraint violation: {np.linalg.norm(A @ x_opt - b):.2e}")

    # Verify KKT conditions
    kkt_residual = grad_f(x_opt) + A.T @ nu_opt
    print(f"KKT residual ||grad f + A^T nu||: {np.linalg.norm(kkt_residual):.2e}")

demo_equality_constrained()

Newton for Equality-Constrained Optimization
Iter 0: f = -9.773477e-01, lambda = 1.83e+00, ||Ax-b|| = 1.78e-15, eta = 1.0000
Converged at iteration 1

Optimal x (first 5): [ 1.12463372  1.85995114 -1.43299828  0.92893075  0.8391533 ]
Optimal multipliers: [-0.75215757  0.53110276  0.08493105]
Final constraint violation: 1.78e-15
KKT residual ||grad f + A^T nu||: 2.77e-15
